<a href="https://colab.research.google.com/github/MakkaiNandor/IntelligensModszerekTantargy/blob/main/NeuralisHaloForditasaJavaScriptKodda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

Adatok betöltése és kiválasztása: az egyszerűség kedvéért mindössze három oszlopot választunk ki.

In [2]:
data = pd.read_csv('http://www.biointelligence.hu/mi/fuel_data.txt', header=0, sep='\t')

selected_data = pd.DataFrame()
selected_data['starttemp'] = data['starttemp']
selected_data['endtemp'] = data['endtemp']
selected_data['speed'] = data['speed']
train_data = np.array(selected_data)
train_labels = np.array([ [y] for y in data['avg.cons.']])

A modell definiálása és tanítása

In [3]:
class ConsumptionNet(nn.Module):
    def __init__(self):
        super(ConsumptionNet, self).__init__()
        self.hidden = nn.Linear(3, 10)
        self.out = nn.Linear(10, 1) 

    def forward(self, x):
        x = torch.relu(self.hidden(x))
        x = self.out(x)
        return x

train_dataset = torch.utils.data.TensorDataset(
    torch.Tensor(train_data), torch.Tensor(train_labels) )
trainloader = torch.utils.data.DataLoader(
    train_dataset, shuffle=True, batch_size=1)

net = ConsumptionNet()
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=1e-5)

running_loss = 0.0
running_n = 0

for epoch in range(1000):  
  for inputs, targets in trainloader:
    optimizer.zero_grad()
    
    outputs = net(inputs)

    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    running_n = running_n + 1

  if epoch % 100 == 0:
    print('epoch %d, loss: %.3f' % (epoch, running_loss / running_n))
  running_loss = 0.0
  running_n = 0   

epoch 0, loss: 15.028
epoch 100, loss: 2.992
epoch 200, loss: 2.484
epoch 300, loss: 1.788
epoch 400, loss: 1.160
epoch 500, loss: 0.741
epoch 600, loss: 0.550
epoch 700, loss: 0.473
epoch 800, loss: 0.440
epoch 900, loss: 0.433


In [4]:
for p in net.parameters():
  print(p)

Parameter containing:
tensor([[ 0.0586, -0.1018,  0.4327],
        [-0.3288,  0.3255, -0.4252],
        [-0.3280,  0.1263, -0.3150],
        [ 0.2268,  0.5603,  0.4575],
        [-0.5676, -0.0334, -0.1900],
        [-0.0802,  0.4612,  0.0396],
        [-0.0978,  0.1365,  0.2340],
        [-0.3384, -0.2432, -0.0587],
        [ 0.9318,  0.2085, -0.7663],
        [-0.0285, -0.4831,  0.1085]], requires_grad=True)
Parameter containing:
tensor([ 0.3481,  0.4639, -0.1518, -0.4470, -0.0642,  0.5055,  0.3584,  0.3559,
        -0.0065, -0.4696], requires_grad=True)
Parameter containing:
tensor([[-0.0670, -0.0885,  0.1640,  0.1604,  0.1726, -0.1883,  0.0881,  0.2505,
          1.0888, -0.2527]], requires_grad=True)
Parameter containing:
tensor([0.2440], requires_grad=True)


Tanulmányozzuk a következő kódot és mentsük le egy egyszerű szöveg ("plain text") formátumú fájlba javascript_pelda.html néven. Nyissuk meg a fájlt egy webböngészőben!

    <html>
    <head>
    <title>
      Neurális hálózat fordítása JavaScript kóddá
    </title>
    </head>

    <script type="text/javascript">  
      function estimate_cons() {
        /* itt fogjuk megvalósítani a neurális háló által végzett számításokat */
        window.alert("Még nincs implementálva")
      }
    </script>  

    <body>
    <h1>Fogyasztás becslése neurális hálóval<h1>

    <form name="input_data">
    <table>
    <tr>
      <td>Hőmérsékelt induláskor:</td>
      <td><input type="text" name="temp1" size=3></td>
    </tr>
    <tr>
      <td>Hőmérsékelt érkezéskor:</td>
      <td><input type="text" name="temp2" size=3></td>
    </tr>
    <tr>
      <td>Átlagsebesség:</td>
      <td><input type="text" name="speed" size=3></td>
    </tr>
    <tr>
      <td></td>  
      <td><input type="button" onclick="estimate_cons()" value="Számold ki!"></td>
    </tr>
    </table>
    </form>
    </body>
    </html>

A neurális háló paramétereinek ismeretében akár "kézzel" is megírhatjuk azt a kódot, amely ugyanazon számításokat elvégzi, amit a neurális háló is végez. 

Ha például az alábbi paraméterértékek adódtak a tanítás során:

    Parameter containing:
    tensor([[ 0.2509, -0.2061,  0.1073],
            ... ], requires_grad=True)
    Parameter containing:
    tensor([-0.2061,  ...], requires_grad=True)
    ...

akkor ezek alapján a rejtett réteg első unitjának aktiválása az alábbi JavaScript kóddal számolható:
    
    h1 = 0.2509*input1 - 0.2061*input2 - 0.1073*input3 - 0.2061;
    if (h1 < 0) { h1 = 0; }   /* ReLU */


Hasonlóképpen megírhatjuk a kimeneti réteg számításait elvégző kódot is. A kimeneti réteg paraméterei:

    Parameter containing:
    tensor([[-0.1407, -0.0554,  0.1907,  0.1611,  0.6042,  0.1442, -0.0315,  0.3453,
              0.6535, -0.2172]], requires_grad=True)
    Parameter containing:
    tensor([0.3992], requires_grad=True)

Ezek alapján a kimenetet megvalósító kód:

        out = -0.1407*h1 - 0.0554*h2 + 0.1907*h3 + 0.1611*h4 + 0.6042*h5 + 0.1442*h6 - 0.0315*h7 + 0.3453*h8 + 0.6535*h9 - 0.2172*h10 + 0.3992;


Realisztikus méretű neurális hálók célplattformra történő fordítása tipikusan automatizált módon történik, például írunk egy függvényt Python-ban, amely bementként megkapja a tanított neurális háló paramétereit, és visszaadja a számításokat elvégző JavaScript kódot.

Összességében az alábbi HTML/JavaScript kód adódik:

    <html>
    <head>
    <title>
      Neurális hálózat fordítása JavaScript kóddá
    </title>
    </head>

    <script type="text/javascript">  
      function estimate_cons() {
        input1 = window.document.input_data.temp1.value;
        input2 = window.document.input_data.temp2.value;
        input3 = window.document.input_data.speed.value;

        h1  =  0.2509*input1 - 0.2061*input2 - 0.1073*input3 - 0.2061;
        h2  = -0.2059*input1 - 0.2937*input2 - 0.1009*input3 + 0.4432;
        h3  =  0.1150*input1 + 0.2625*input2 - 0.3610*input3 + 0.1330;
        h4  =  0.6465*input1 - 0.2293*input2 + 0.4714*input3 - 0.2794;
        h5  =  0.3817*input1 + 0.4118*input2 - 0.5667*input3 - 0.1213;
        h6  = -0.3119*input1 - 0.4398*input2 + 0.0307*input3 - 0.1389;
        h7  =  0.0666*input1 + 0.1040*input2 + 0.1719*input3 - 0.5187;
        h8  =  0.3125*input1 + 0.1885*input2 - 0.3690*input3 + 0.2188;
        h9  =  0.1097*input1 + 0.7307*input2 - 0.4844*input3 - 0.4756;
        h10 = -0.3408*input1 - 0.2709*input2 + 0.0361*input3 - 0.1425;
        
        if (h1 < 0) { h1 = 0; }
        if (h2 < 0) { h2 = 0; }
        if (h3 < 0) { h3 = 0; }
        if (h4 < 0) { h4 = 0; }
        if (h5 < 0) { h5 = 0; }
        if (h6 < 0) { h6 = 0; }
        if (h7 < 0) { h7 = 0; }
        if (h8 < 0) { h8 = 0; }
        if (h9 < 0) { h9 = 0; }
        if (h10 < 0) { h10 = 0; }

        out = -0.1407*h1 - 0.0554*h2 + 0.1907*h3 + 0.1611*h4 + 0.6042*h5 + 0.1442*h6 - 0.0315*h7 + 0.3453*h8 + 0.6535*h9 - 0.2172*h10 + 0.3992;

        window.alert("Becsült fogyasztás: "+out)
      }
    </script>  

    <body>
    <h1>Fogyasztás becslése neurális hálóval<h1>

    <form name="input_data">
    <table>
    <tr>
      <td>Hőmérsékelt induláskor:</td>
      <td><input type="text" name="temp1" size=3> fok</td>
    </tr>
    <tr>
      <td>Hőmérsékelt érkezéskor:</td>
      <td><input type="text" name="temp2" size=3> fok</td>
    </tr>
    <tr>
      <td>Átlagsebesség:</td>
      <td><input type="text" name="speed" size=3> km/h</td>
    </tr>
    <tr>
      <td></td>  
      <td><input type="button" onclick="estimate_cons()" value="Számold ki!"></td>
    </tr>
    </table>
    </form>
    </body>
    </html>